In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer




[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lildreamz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:

def bagwords (df):
       
    
    listname = ['artist_genres', 'artist_name', 'album_name', 'song_name']
    
  
    dicname = {name: [] for name in listname}
    dicname['song_id'] = []
    
    listid = []
    
    dicdf = {}
    vol = {}
    print(df.shape)
 
        
    for name in listname:
        
        for idx in range(df[name].size):
            
            l = df[name][idx]
          
            r = names_to_words(l)
            
            dicname[name].append(r) 
            
      
            
        
        vectorizer = CountVectorizer(analyzer='word',max_features=30)
        feature = vectorizer.fit_transform(dicname[name]).toarray().tolist()
        vol[name] = vectorizer.get_feature_names()
        
        
        dicdf[name] = pd.DataFrame({name: feature}) 
        
        dicdf[name] = dicdf[name][name].apply(pd.Series)
        ### dicdf[name] already lose column tag name, but change into 0 1 2 
        
        l = len(dicdf[name].columns)
        
        dicdf[name].columns = [(name + '_' +vol[name][x]) for x in range(l)]
        
        print('good')
     
   
    
    for idx in range(df['song_id'].size):
        sid = df['song_id'][idx]
        listid.append(sid)
       
            
        dicdf['song_id'] = pd.DataFrame({'song_id':listid}) 

        
            
            
        
    result = pd.concat(dicdf.values(), axis =1)
    
    return result
    


def names_to_words(names):
    words = re.sub("[^a-zA-Z0-9]"," ",names).lower().split()
    
    words = [i for i in words if i not in set(stopwords.words("english"))]
    ## Need join as string for countvectorizer!
    return (" ".join(words))

def reduce_genres(gen):
    genre = re.sub("[^a-zA-Z0-9]"," ",gen).lower().split()
    genre = [i for i in genre if i not in set(stopwords.words("english"))]
    mode1 = str(stats.mode(genre)).split('[')[1].split(']')[0]
    return mode1
    



In [3]:
print('Start!')

## general dataframe transform
df = pd.read_csv('/home/lildreamz/Programming/jupyter_notebooks/spotifyAnalysis/1995.csv', sep='\t')

df = df.dropna()



Start!


In [4]:
df = df.drop(['album_genres'],axis =1 )

df.head()



,Unnamed: 0,popularity,song_id,artist_id,album_id,song_name,artist_name,album_name,explicit,disc_number,...,track_href,analysis_url,duration_ms,time_signature,acousticness,instrumentalness,artist_genres,artist_popularity,album_popularity,album_release_date
0,0,73,5wj4E6IsrVtn8IBJQOd0Cl,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Wonderwall - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,False,1,...,https://api.spotify.com/v1/tracks/5wj4E6IsrVtn...,https://api.spotify.com/v1/audio-analysis/5wj4...,258733,4,0.000453,0.000000,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02
1,1,64,7CVYxHq1L0Z4G84jTDS6Jl,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Don't Look Back In Anger - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,False,1,...,https://api.spotify.com/v1/tracks/7CVYxHq1L0Z4...,https://api.spotify.com/v1/audio-analysis/7CVY...,289559,4,0.059600,0.000002,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02
2,2,63,40bynawzslg9U7ACq07fAj,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Champagne Supernova - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,False,1,...,https://api.spotify.com/v1/tracks/40bynawzslg9...,https://api.spotify.com/v1/audio-analysis/40by...,448438,4,0.470000,0.063000,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02
3,3,46,5hDBfsFoLFkIXt9xDi3th4,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,She's Electric - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,False,1,...,https://api.spotify.com/v1/tracks/5hDBfsFoLFkI...,https://api.spotify.com/v1/audio-analysis/5hDB...,219799,4,0.164000,0.000002,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02
4,4,45,69R0w4lQDO7AFx7D2ytRcV,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Some Might Say - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,False,1,...,https://api.spotify.com/v1/tracks/69R0w4lQDO7A...,https://api.spotify.com/v1/audio-analysis/69R0...,327839,4,0.162000,0.013000,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02


In [5]:
df['explicit'] = df['explicit'].map( {True: 1, False: 0} ).astype(int) 
df.head()

,Unnamed: 0,popularity,song_id,artist_id,album_id,song_name,artist_name,album_name,explicit,disc_number,...,track_href,analysis_url,duration_ms,time_signature,acousticness,instrumentalness,artist_genres,artist_popularity,album_popularity,album_release_date
0,0,73,5wj4E6IsrVtn8IBJQOd0Cl,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Wonderwall - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,1,...,https://api.spotify.com/v1/tracks/5wj4E6IsrVtn...,https://api.spotify.com/v1/audio-analysis/5wj4...,258733,4,0.000453,0.000000,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02
1,1,64,7CVYxHq1L0Z4G84jTDS6Jl,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Don't Look Back In Anger - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,1,...,https://api.spotify.com/v1/tracks/7CVYxHq1L0Z4...,https://api.spotify.com/v1/audio-analysis/7CVY...,289559,4,0.059600,0.000002,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02
2,2,63,40bynawzslg9U7ACq07fAj,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Champagne Supernova - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,1,...,https://api.spotify.com/v1/tracks/40bynawzslg9...,https://api.spotify.com/v1/audio-analysis/40by...,448438,4,0.470000,0.063000,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02
3,3,46,5hDBfsFoLFkIXt9xDi3th4,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,She's Electric - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,1,...,https://api.spotify.com/v1/tracks/5hDBfsFoLFkI...,https://api.spotify.com/v1/audio-analysis/5hDB...,219799,4,0.164000,0.000002,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02
4,4,45,69R0w4lQDO7AFx7D2ytRcV,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Some Might Say - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,1,...,https://api.spotify.com/v1/tracks/69R0w4lQDO7A...,https://api.spotify.com/v1/audio-analysis/69R0...,327839,4,0.162000,0.013000,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02


In [6]:
z = df['popularity'].quantile(0.8)
df['class'] = df['popularity'].apply(lambda x: 1 if x >= z else 0)
df.head()

,Unnamed: 0,popularity,song_id,artist_id,album_id,song_name,artist_name,album_name,explicit,disc_number,...,analysis_url,duration_ms,time_signature,acousticness,instrumentalness,artist_genres,artist_popularity,album_popularity,album_release_date,class
0,0,73,5wj4E6IsrVtn8IBJQOd0Cl,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Wonderwall - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,1,...,https://api.spotify.com/v1/audio-analysis/5wj4...,258733,4,0.000453,0.000000,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02,1
1,1,64,7CVYxHq1L0Z4G84jTDS6Jl,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Don't Look Back In Anger - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,1,...,https://api.spotify.com/v1/audio-analysis/7CVY...,289559,4,0.059600,0.000002,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02,1
2,2,63,40bynawzslg9U7ACq07fAj,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Champagne Supernova - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,1,...,https://api.spotify.com/v1/audio-analysis/40by...,448438,4,0.470000,0.063000,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02,1
3,3,46,5hDBfsFoLFkIXt9xDi3th4,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,She's Electric - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,1,...,https://api.spotify.com/v1/audio-analysis/5hDB...,219799,4,0.164000,0.000002,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02,1
4,4,45,69R0w4lQDO7AFx7D2ytRcV,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Some Might Say - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,1,...,https://api.spotify.com/v1/audio-analysis/69R0...,327839,4,0.162000,0.013000,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02,1


In [10]:
df['artist_genres'].apply(lambda x: x)


0       ['britpop', 'madchester', 'permanent wave', 'r...
1       ['britpop', 'madchester', 'permanent wave', 'r...
2       ['britpop', 'madchester', 'permanent wave', 'r...
3       ['britpop', 'madchester', 'permanent wave', 'r...
4       ['britpop', 'madchester', 'permanent wave', 'r...
5       ['britpop', 'madchester', 'permanent wave', 'r...
6       ['britpop', 'madchester', 'permanent wave', 'r...
7       ['britpop', 'madchester', 'permanent wave', 'r...
8       ['britpop', 'madchester', 'permanent wave', 'r...
9       ['britpop', 'madchester', 'permanent wave', 'r...
10      ['britpop', 'madchester', 'permanent wave', 'r...
11      ['britpop', 'madchester', 'permanent wave', 'r...
12      ['britpop', 'madchester', 'permanent wave', 'r...
13      ['britpop', 'madchester', 'permanent wave', 'r...
14      ['britpop', 'madchester', 'permanent wave', 'r...
15      ['britpop', 'madchester', 'permanent wave', 'r...
16      ['britpop', 'madchester', 'permanent wave', 'r...
17      ['brit

In [7]:
df = df[(df.astype(str)['artist_genres'] != '[]')].reset_index()
df.head()

,index,Unnamed: 0,popularity,song_id,artist_id,album_id,song_name,artist_name,album_name,explicit,...,analysis_url,duration_ms,time_signature,acousticness,instrumentalness,artist_genres,artist_popularity,album_popularity,album_release_date,class
0,0,0,73,5wj4E6IsrVtn8IBJQOd0Cl,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Wonderwall - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,...,https://api.spotify.com/v1/audio-analysis/5wj4...,258733,4,0.000453,0.000000,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02,1
1,1,1,64,7CVYxHq1L0Z4G84jTDS6Jl,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Don't Look Back In Anger - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,...,https://api.spotify.com/v1/audio-analysis/7CVY...,289559,4,0.059600,0.000002,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02,1
2,2,2,63,40bynawzslg9U7ACq07fAj,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Champagne Supernova - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,...,https://api.spotify.com/v1/audio-analysis/40by...,448438,4,0.470000,0.063000,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02,1
3,3,3,46,5hDBfsFoLFkIXt9xDi3th4,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,She's Electric - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,...,https://api.spotify.com/v1/audio-analysis/5hDB...,219799,4,0.164000,0.000002,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02,1
4,4,4,45,69R0w4lQDO7AFx7D2ytRcV,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Some Might Say - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,...,https://api.spotify.com/v1/audio-analysis/69R0...,327839,4,0.162000,0.013000,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02,1


In [8]:
df['reduced_genres'] = df['artist_genres'].apply(lambda x: reduce_genres(x))


/home/lildreamz/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [9]:
df.tail()

,index,Unnamed: 0,popularity,song_id,artist_id,album_id,song_name,artist_name,album_name,explicit,...,duration_ms,time_signature,acousticness,instrumentalness,artist_genres,artist_popularity,album_popularity,album_release_date,class,reduced_genres
9535,9993,9993,14,1kWBr7sIS7zJiYxxKADwJq,2OpqcUtj10HHvGG6h9VYC5,1x4TEJ7zcd5YCOM9cOR8g6,Will You Travel Down This Road With Me,George Jones,One,0,...,190133,4,0.4440,0.000002,"['country', 'country gospel', 'country rock', ...",62,17,1995-01-01,0,'country'
9536,9994,9994,13,25dhHNVxCAv9sMQuuVDKXL,7BaZ6eyG6duh3m6sfKIoe7,7aoK7Ur4zqvoanHVQKcXxf,Sax Machine,Axiom Funk,Funkcronomicon,0,...,465800,4,0.0134,0.691000,['p funk'],11,15,1995-01-01,0,'funk'
9537,9995,9995,20,2V0EnNxIKBrvyAxqPJQvJV,4v0R1feRiuCDch7aAheVhY,4CTNr6rdgCXDH6vjMOKCC3,My Favourite Things - Instrumental,John McLaughlin,After The Rain,0,...,376400,3,0.5970,0.325000,"['british jazz', 'canterbury scene', 'contempo...",47,17,1995-01-01,0,'jazz'
9538,9996,9996,22,2c3ozQeEi7VVtxvuWjVJlV,6uhyakkTnWoxbEKn9pIc6V,0dP30ZOiJx3LuK0DhVZeLe,Main theme,John Ottman,The Usual Suspects,0,...,220973,4,0.5900,0.889000,"['classic soundtrack', 'epicore', 'scorecore',...",45,19,1995,0,'soundtrack'
9539,9997,9997,44,2vvh3ppgLLCYaiZJq0ElKI,4li4dx5mFgZlMVeHlARBHP,6vS9HeASVc9y02B86OYBw8,Gidelim Buralardan,Nazan Öncel,Göç,0,...,278680,4,0.1330,0.000000,"['turkish folk', 'turkish jazz', 'turkish pop'...",49,37,1995-11-15,1,'turkish'


In [10]:
df['year'] = [x.split('-')[0] for x in df['album_release_date']]

#df.to_csv('bigdiwithyear.csv', sep='\t')

In [11]:
df.head()

,index,Unnamed: 0,popularity,song_id,artist_id,album_id,song_name,artist_name,album_name,explicit,...,time_signature,acousticness,instrumentalness,artist_genres,artist_popularity,album_popularity,album_release_date,class,reduced_genres,year
0,0,0,73,5wj4E6IsrVtn8IBJQOd0Cl,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Wonderwall - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,...,4,0.000453,0.000000,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02,1,'britpop',1995
1,1,1,64,7CVYxHq1L0Z4G84jTDS6Jl,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Don't Look Back In Anger - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,...,4,0.059600,0.000002,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02,1,'britpop',1995
2,2,2,63,40bynawzslg9U7ACq07fAj,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Champagne Supernova - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,...,4,0.470000,0.063000,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02,1,'britpop',1995
3,3,3,46,5hDBfsFoLFkIXt9xDi3th4,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,She's Electric - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,...,4,0.164000,0.000002,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02,1,'britpop',1995
4,4,4,45,69R0w4lQDO7AFx7D2ytRcV,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Some Might Say - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,...,4,0.162000,0.013000,"['britpop', 'madchester', 'permanent wave', 'r...",79,67,1995-10-02,1,'britpop',1995


In [6]:
df.shape

(10000, 33)

In [7]:
df.head()

,Unnamed: 0,popularity,song_id,artist_id,album_id,song_name,artist_name,album_name,explicit,disc_number,...,analysis_url,duration_ms,time_signature,acousticness,instrumentalness,artist_genres,artist_popularity,album_genres,album_popularity,album_release_date
0,0,73,5wj4E6IsrVtn8IBJQOd0Cl,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Wonderwall - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,False,1,...,https://api.spotify.com/v1/audio-analysis/5wj4...,258733,4,0.000453,0.000000,"['britpop', 'madchester', 'permanent wave', 'r...",79,[],67,1995-10-02
1,1,64,7CVYxHq1L0Z4G84jTDS6Jl,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Don't Look Back In Anger - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,False,1,...,https://api.spotify.com/v1/audio-analysis/7CVY...,289559,4,0.059600,0.000002,"['britpop', 'madchester', 'permanent wave', 'r...",79,[],67,1995-10-02
2,2,63,40bynawzslg9U7ACq07fAj,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Champagne Supernova - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,False,1,...,https://api.spotify.com/v1/audio-analysis/40by...,448438,4,0.470000,0.063000,"['britpop', 'madchester', 'permanent wave', 'r...",79,[],67,1995-10-02
3,3,46,5hDBfsFoLFkIXt9xDi3th4,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,She's Electric - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,False,1,...,https://api.spotify.com/v1/audio-analysis/5hDB...,219799,4,0.164000,0.000002,"['britpop', 'madchester', 'permanent wave', 'r...",79,[],67,1995-10-02
4,4,45,69R0w4lQDO7AFx7D2ytRcV,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Some Might Say - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,False,1,...,https://api.spotify.com/v1/audio-analysis/69R0...,327839,4,0.162000,0.013000,"['britpop', 'madchester', 'permanent wave', 'r...",79,[],67,1995-10-02


In [88]:
df1 = bagwords( df )

df1.head()

(9540, 36)
good
good
good
good


,artist_genres_alternative,artist_genres_blues,artist_genres_classic,artist_genres_coast,artist_genres_contemporary,artist_genres_country,artist_genres_dance,artist_genres_folk,artist_genres_funk,artist_genres_gangster,...,song_name_op,song_name_que,song_name_remastered,song_name_single,song_name_song,song_name_suite,song_name_te,song_name_time,song_name_version,song_id
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,5wj4E6IsrVtn8IBJQOd0Cl
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,7CVYxHq1L0Z4G84jTDS6Jl
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,40bynawzslg9U7ACq07fAj
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,5hDBfsFoLFkIXt9xDi3th4
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,69R0w4lQDO7AFx7D2ytRcV


In [89]:
df = df.merge(df1, on='song_id', how='outer')


In [91]:

df.head()

,index,Unnamed: 0,popularity,song_id,artist_id,album_id,song_name,artist_name,album_name,explicit,...,song_name_one,song_name_op,song_name_que,song_name_remastered,song_name_single,song_name_song,song_name_suite,song_name_te,song_name_time,song_name_version
0,0,0,73,5wj4E6IsrVtn8IBJQOd0Cl,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Wonderwall - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,...,0,0,0,1,0,0,0,0,0,0
1,1,1,64,7CVYxHq1L0Z4G84jTDS6Jl,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Don't Look Back In Anger - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,...,0,0,0,1,0,0,0,0,0,0
2,2,2,63,40bynawzslg9U7ACq07fAj,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Champagne Supernova - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,...,0,0,0,1,0,0,0,0,0,0
3,3,3,46,5hDBfsFoLFkIXt9xDi3th4,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,She's Electric - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,...,0,0,0,1,0,0,0,0,0,0
4,4,4,45,69R0w4lQDO7AFx7D2ytRcV,2DaxqgrOhkeH0fpeiQq2f4,6ozp33PI3p9AdddB6ZL3xQ,Some Might Say - Remastered,Oasis,(What's The Story) Morning Glory? (Deluxe Edit...,0,...,0,0,0,1,0,0,0,0,0,0


In [92]:
df = df.drop(['Unnamed: 0', 'song_id', 'artist_id','album_id','song_name',
            'artist_name','album_name','uri', 'type', 'track_href',
            'analysis_url','artist_genres','album_release_date','popularity',
            'class','index','reduced_genres'],axis=1)

In [93]:
df.head()

,explicit,disc_number,track_number,tempo,key,loudness,mode,speechiness,liveness,valence,...,song_name_one,song_name_op,song_name_que,song_name_remastered,song_name_single,song_name_song,song_name_suite,song_name_te,song_name_time,song_name_version
0,0,1,3,174.530,2,-4.918,1,0.0381,0.234,0.478,...,0,0,0,1,0,0,0,0,0,0
1,0,1,4,162.978,0,-4.087,1,0.0592,0.126,0.335,...,0,0,0,1,0,0,0,0,0,0
2,0,1,12,150.272,2,-4.625,1,0.0388,0.364,0.120,...,0,0,0,1,0,0,0,0,0,0
3,0,1,9,125.413,9,-4.288,1,0.0475,0.636,0.526,...,0,0,0,1,0,0,0,0,0,0
4,0,1,7,108.954,0,-3.115,0,0.0896,0.285,0.072,...,0,0,0,1,0,0,0,0,0,0


{'Unnamed: 0': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12,
  13: 13,
  14: 14,
  15: 15,
  16: 16,
  17: 17,
  18: 18,
  19: 19,
  20: 20,
  21: 21,
  22: 22,
  23: 23,
  24: 24,
  25: 25,
  26: 26,
  27: 27,
  28: 28,
  29: 29,
  30: 30,
  31: 31,
  32: 32,
  33: 33,
  34: 34,
  35: 35,
  36: 36,
  37: 37,
  38: 38,
  39: 39,
  40: 40,
  41: 41,
  42: 42,
  43: 43,
  44: 44,
  45: 45,
  46: 46,
  47: 47,
  48: 48,
  49: 49,
  50: 50,
  51: 51,
  52: 52,
  53: 53,
  54: 54,
  55: 55,
  56: 56,
  57: 57,
  58: 58,
  59: 59,
  60: 60,
  61: 61,
  62: 62,
  63: 63,
  64: 64,
  65: 65,
  66: 66,
  67: 67,
  68: 68,
  69: 69,
  70: 70,
  71: 71,
  72: 72,
  73: 73,
  74: 74,
  75: 75,
  76: 76,
  77: 77,
  78: 78,
  79: 79,
  80: 80,
  81: 81,
  82: 82,
  83: 83,
  84: 84,
  85: 85,
  86: 86,
  87: 87,
  88: 88,
  89: 89,
  90: 90,
  91: 91,
  92: 92,
  93: 93,
  94: 94,
  95: 95,
  96: 96,
  97: 97,
  98: 98,
  99: 99,
  100:

In [ ]:
### bag of words: text items vectorization

df1 = bagwords( df )

df = df.merge(df1, on='song_id', how='outer')

df.shape


df = df.drop(['Unnamed: 0', 'song_id', 'artist_id','album_id','song_name',
            'artist_name','album_name','uri', 'type', 'track_href',
            'analysis_url','artist_genres','album_release_date','popularity',
            'class','index','reduced_genres'],axis=1)   

#df.to_csv('bigdichorddiagram.csv', sep='\t')

#df[(df['popularity'] >= df['popularity'].quantile(0.8))]['loudness'].mean()#value_counts(dropna=False)



#training
Y = df['class'].values

X = df.values     

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)


#clf = XGBClassifier()
#clf = RandomForestRegressor(n_estimators=200,max_features=18)

clf  =  XGBClassifier(
        eval_metric = 'accuracy',
        learning_rate = 0.1,
        n_estimators = 100,
        max_depth = 3,
        subsample = 0.9,
        colsample_bytree = 0.9,
        silent = False
        )

#clf = RandomForestClassifier(n_estimators=100,max_features=17)  

#clf = svm.SVC()        
        
clf.fit(X_train,Y_train)

print('good')

importance = clf.feature_importances_

dfi = pd.DataFrame(importance, index=df.columns, columns=["Importance"])
dfi = dfi.sort_values(['Importance'],ascending=False)
print(dfi)
#dfi.plot(kind='bar',color='Purple')



Ascores_Train = cross_val_score(clf, X_train, Y_train, cv=5)
Ascores_Test = clf.score(X_test,Y_test)


print('lala')
print(Ascores_Train.mean()) 
print(Ascores_Train.std()) 
print(Ascores_Test) 


Ascores_Train = cross_val_score(clf, X_train, Y_train, cv=5)
Ascores_Test = clf.score(X_test,Y_test)


print('lala')
print(Ascores_Train.mean()) 
print(Ascores_Train.std()) 
print(Ascores_Test) 




# Predicted_Train = cross_val_predict(clf, X_train, Y_train, cv=5)
# MSE_Train = mean_squared_error(Y_train, Predicted_Train)
# print('MSE for training data is:')
# print(MSE_Train)


# Predicted_Test = clf.predict(X_test)
# MSE_Test = mean_squared_error(Y_test, Predicted_Test)
# print('MSE for test data is:')
# print(MSE_Test)



# Predicted_Train = cross_val_predict(clf, X_train, Y_train, cv=5)
# MSE_Train = mean_squared_error(Y_train, Predicted_Train)
# print('MSE for training data is:')
# print(MSE_Train)


# Predicted_Test = clf.predict(X_test)
# MSE_Test = mean_squared_error(Y_test, Predicted_Test)
# print('MSE for test data is:')
# print(MSE_Test)

